In [3]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.datasets
import sklearn.linear_model
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
states_data = pd.read_csv('4.18states.csv')
counties = pd.read_csv('abridged_couties.csv')
